In [3]:
import pandas as pd
import requests
import json
import re

In [2]:
with open("C:\\Users\\Paul Cassidy\\Documents\\AoC_QBO_Integration\\RawData\\GL_2021-07-01_2021-07-26.json") as f:
    gl_1 = json.load(f)

In [4]:
with open("..\\RawData\\GL_06-29_07-09_minorversion=3.json") as f:
    gl_2 = json.load(f)

In [4]:
with open("..\\RawData\\GL_2021-07-01_2021-07-26.json") as f:
    gl = json.load(f)

In [5]:
with open("..\\RawData\\GL_2021-07-01_2021_07_26_NMV.json") as f:
    gl_nmv = json.load(f)

In [6]:
with open("..\\RawData\\GeneralLedgerReal.json") as f:
    gl_real = json.load(f)

In [37]:
with open("C:\\Users\\Paul Cassidy\\Documents\\AoC_QBO_Integration\\RawData\\GL_06-29_07-09.json") as f:
    gl_3 = json.load(f)

In [49]:
def parse_record_gl(data):
    """
    Iterate through ColData or Header.ColData data frame. 
    Check what kind of data each data frame contains.
    Parse each record in data frame accordingly and return values.
    """
    # Regular expression matching date format
    date_rex = re.compile("^[0-9]{4}-[0-9]{2}-[0-9]{2}$")

    # Check which ColData data frame is being parsed
    for row in range(data.shape[0]):
        # Transaction detail data frame
        
        if re.search(date_rex, data['value'][0]):
            vals = ["", "", "",]
            # vals[0] = AccountName
            # vals[1] = AccountID
            # vals[2] = Beginning Balance
            for row in range(data.shape[0]):
                if 'id' in data:
                    val    = data['value'][row]
                    val_id = data['id'][row]
                    vals.append(val)
                    vals.append(val_id)
                else:
                    val = data['value'][row]
                    vals.append(val)
                    vals.append(float(NaN))
            return vals

        # Beginning balance data frame
        elif data['value'][0] == "Beginning Balance":
            vals = ["", ""]
            begbal = data['value'][len(data)-1]
            vals.append(begbal)
            return vals

        # Name data frame
        else:
            # Check for absent Account ID and impute NULL if return False
            if 'id' not in data:
                vals = [data['value'][0], float("NaN")]
            else:
                vals = [data['value'][0], data['id'][0]]
                
            return vals

def assign_col_data(coldata, end):
    """
    Assign values parsed from transaction detail object to global row_dict var.
    """
    for i in range(3):
        if not coldata[i]:
            coldata[i] = end[i]
        else:
            pass
    
    return coldata

def get_col_names(data):
    first_col_tbl = pd.json_normalize(data['Columns'])
    col_names_tbl = pd.json_normalize(first_col_tbl['Column'][0])
    count = 0
    col_names = ["AccountName", "AccountNameID", "BeginningBalance"]
    for row in range(col_names_tbl.shape[0]):
        col_names.append(col_names_tbl['ColTitle'][row])
        col_names.append(str(col_names_tbl['ColTitle'][row]) + "ID")
        count += 2
    col_names = [col_name.replace(" ", "") for col_name in col_names]
    col_names.insert(0, count + 3)
    return col_names

def impute_null(lst):
    """
    Impute null values where no transaction detail exists.
    """
    for i in range(len(lst)):
        if not lst[i]:
            lst[i] = float("NaN")
        elif type(lst[i]) == float:
            lst[i] = float("NaN")
    return lst

# Initialize container variable for row data
# Will act as global variable for crawler function to store values
# throughout recursions.
row_dict = {}

def json_crawler_gl(data):
    """
    Iterate through ColData or Header.ColData data frame. 
    Check what kind of data each data frame contains.
    Parse each record in data frame accordingly and return values.
    """
    global row_dict

    # If data is dictionary returned by json.load(), then flatten it to initial pandas data frame
    if isinstance(data, dict):
        col_names = get_col_names(data)
        data = pd.json_normalize(data)
        return_value = col_names + json_crawler_gl(data)
        return return_value 
    
    # Crawl through JSON file, normalizing by different keys to find account data and save to lst
    else:
        # Initialize list to hold account data objects
        lst = []
        
        for row in range(data.shape[0]):
            # Normalize by 'Header.ColData'
            try:
                record_data = pd.json_normalize(data['Header.ColData'][row])
                row_dict = impute_null(parse_record_gl(record_data))
                lst.append(row_dict)
                # print(lst)
            # except Exception as e:
            #     print("Line 121", e)
            except:
                pass
            # Normalize by 'ColData'
            try:
                record_data = pd.json_normalize(data['ColData'][row])
                if row == 0:
                    row_dict.append(parse_record_gl(record_data)[2])
                else:
                    # print(parse_record_gl(record_data), row_dict)
                    # print(assign_col_data(parse_record_gl(record_data), row_dict))
                    new_dict = list(impute_null(assign_col_data(parse_record_gl(record_data), row_dict)))
                    lst.append(new_dict)
            except:
                pass
            # Normalize by 'Rows.Row'
            try:
                row_data = pd.json_normalize(data['Rows.Row'][row])
                # Capture lst and return value before recursive function call
                return_value = json_crawler_gl(row_data)
                lst = lst + return_value
            except:
                pass
        return lst

def extract_col_names(lst):
    num_of_col = lst.pop(0)
    col_names  = lst[:(num_of_col)]
    del lst[:(num_of_col)]
    return col_names
    
def norm_length(lst, col_names):
    for i in range(len(lst)):
        if len(lst[i]) != len(col_names):
            for j in range(len(col_names) - len(lst[i])):
                lst[i].append(float("NaN"))
    return lst

def gl_return_output(data, folder, filename):
    lst       = json_crawler_gl(data)
    col_names = extract_col_names(lst)
    output    = norm_length(lst, col_names)
    df        = pd.DataFrame(output, columns=col_names)
    final_obj = pd.DataFrame.to_dict(df, orient="records")
    return final_obj
    


In [50]:
gl_return_output(gl_2, "../Output/", "GL_MV3_SJLSDR")

[{'AccountName': '1100 Cash',
  'AccountNameID': nan,
  'BeginningBalance': nan,
  'Date': nan,
  'DateID': nan,
  'TransactionType': nan,
  'TransactionTypeID': nan,
  'Num': nan,
  'NumID': nan,
  'Adj': nan,
  'AdjID': nan,
  'Name': nan,
  'NameID': nan,
  'Memo/Description': nan,
  'Memo/DescriptionID': nan,
  'Split': nan,
  'SplitID': nan,
  'Amount': nan,
  'AmountID': nan,
  'Balance': nan,
  'BalanceID': nan},
 {'AccountName': '1101 SJLSDR Operating Acct#2513',
  'AccountNameID': nan,
  'BeginningBalance': nan,
  'Date': nan,
  'DateID': nan,
  'TransactionType': nan,
  'TransactionTypeID': nan,
  'Num': nan,
  'NumID': nan,
  'Adj': nan,
  'AdjID': nan,
  'Name': nan,
  'NameID': nan,
  'Memo/Description': nan,
  'Memo/DescriptionID': nan,
  'Split': nan,
  'SplitID': nan,
  'Amount': nan,
  'AmountID': nan,
  'Balance': nan,
  'BalanceID': nan},
 {'AccountName': '1101.10 Church Available',
  'AccountNameID': '447',
  'BeginningBalance': '69511.88',
  'Date': nan,
  'DateID'

In [ ]:
gl_return_output(gl_3, "../Output/", "GL_MV3_SJLSDR_2")

In [32]:
gl_return_output(gl_nmv, "../Output/", "GL_NMV_JFX")

[19, 'AccountName', 'AccountNameID', 'BeginningBalance', 'Date', 'DateID', 'TransactionType', 'TransactionTypeID', 'Num', 'NumID', 'Name', 'NameID', 'Memo/Description', 'Memo/DescriptionID', 'Split', 'SplitID', 'Amount', 'AmountID', 'Balance', 'BalanceID', ['1100 Cash', '565'], ['1101 Saints JFX Oper. Acct. #2264', 'NULL', '-13922.83'], ['1101.10 Checking Available', '691', '-126471.83'], ['1101.20 Checking Restricted', '587'], ['1101.22 Building Restricted', 'NULL', '3570.33'], ['1101.23 School Support Restricted', '675', '4591.00'], ['1101.24 AOC A/P Restricted', '588', '63.00'], ['101.24A Latin America', '621', '1426.00'], ['101.24C Aid Cent. and East. Europe', '679', '25.00'], ['101.24D Holy Land', '602', '241.00'], ['101.24E Catholic Charities', '652', '1070.00'], ['101.24F Retired Priests', '603', '50.00'], ["101.24G Peter's Pence", '705', '1235.00'], ['101.24H Seminarian Edu.', '604', '100.00'], ['101.24I World MIssion Sunday', '653', '75.00'], ['101.24J CCHD', '605', '865.00'],

In [ ]:
extract_col_names(json_crawler_gl(gl_real))

In [ ]:
output = norm_length(json_crawler_gl(gl_real), extract_col_names(json_crawler_gl(gl_real)))
col_names = extract_col_names(json_crawler_gl(gl_real))
df = pd.DataFrame(output, columns = col_names)
df.dropna(axis="columns", how='all', inplace=True)
df.to_csv("../Output/GL_Adaptive_Columns.csv", index = False)

In [176]:
df.dropna(axis="columns", how='all', inplace=True)

In [178]:
df

,AccountName,AccountNameID,BeginningBalance,Date,TransactionType,TransactionTypeID,Num,Adj,Name,NameID,Memo/Description,Split,SplitID,Amount,Balance
0,1100 Cash,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1101 SJLSDR Operating Acct#2513,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1101.10 Church Available,447,69511.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1101.10 Church Available,447,69511.88,2021-06-30,Deposit,7005,NaN,No,NaN,NaN,St. Francis of Assisi Deposit,3012 Sunday and Holy Day Collections:Sunday Co...,194,1194.00,70705.88
4,1101.10 Church Available,447,69511.88,2021-06-30,Deposit,7004,NaN,No,NaN,NaN,Maternity Deposit,-Split-,NaN,2908.00,73613.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,4450 Maintenance & Building Repairs,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,4459 Janitorial Supplies,94,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,5120 Capital Collections,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,5122 Church Building Fund,129,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
lst = norm_length(json_crawler_gl(gl_real), extract_col_names(json_crawler_gl(gl_real)))
print(lst[3][4])
type(lst[3][4])

nan


float

In [117]:
lst = extract_col_names(json_crawler_gl(gl_real))

In [ ]:
for i in range(len(lst)):
    if len(lst[i]) != 21:
        for j in range(21 - len(lst[i])):
            lst[i].append("")
# print(lst)
for i in range(len(lst)):
    print(len(lst[i]))

In [ ]:
not ""

In [ ]:
first_col_tbl = pd.json_normalize(gl['Columns'])
col_names = pd.json_normalize(first_col_tbl['Column'][0])
col_names

def get_col_names(data):
    first_col_tbl = pd.json_normalize(data['Columns'])
    col_names_tbl = pd.json_normalize(first_col_tbl['Column'][0])
    col_names = ["AccountName", "AccountNameID", "BeginningBalance"]
    for row in range(col_names_tbl.shape[0]):
        col_names.append(col_names_tbl['ColTitle'][row])
        col_names.append(str(col_names_tbl['ColTitle'][row]) + "ID")
    col_names = [col_name.replace(" ", "") for col_name in col_names]
    return col_names

get_col_names(gl)

In [ ]:
json_crawler_gl(gl_2)

In [ ]:
json_crawler_gl(gl)

In [ ]:
json_crawler_gl(gl_nmv)

In [ ]:
json_crawler_gl(gl_real)